In [10]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
%matplotlib inline
import tensorflow as tf

In [4]:
import os
import os.path

In [5]:
validFilePaths = []
for f in os.listdir("data/anomaly_data"):
    filePath = os.path.join("data/anomaly_data", f)
    if os.path.isdir(filePath):
        continue
    if os.stat(filePath).st_size <= 3:
        continue
    validFilePaths.append(filePath)

In [6]:
df = pandas.concat([pandas.read_csv(e) for e in validFilePaths])
df.columns

In [ ]:
# these will require string processing on the column names to work
numericalFeatures = []
categoricalIntegerFeatures = []
categoricalFullFeatures = []

In [ ]:
df_numerical = df[numericalFeatures]
df_cat_num = df[categoricalIntegerFeatures]
df_cat = df[categoricalFullFeatures]

vectorizer = DictVectorizer(sparse = False)
df_cat = vectorizer.fit_transform(df_cat.fillna( 'NA' ).to_dict(orient="records"))

enc = OneHotEncoder(sparse=False)
df_cat_num = enc.fit_(data_cat_num)

In [ ]:
df = pd.concat([df_numerical, df_cat, df_cat_num], axis=1)

In [33]:
np.random.seed(1)
x = np.random.rand(len(df))
mask = x < 0.7
mask1 = np.logical_and(x >= 0.7, x < 0.9) 
mask2 = np.logical_and(x >= 0.9)

In [34]:
df_train = df[mask]
df_validation = df[mask1]
df_test = df[mask2]

In [7]:
NumFeatures = len(df.columns)

In [29]:
def construct(x, layer_size=[10, 10, NumFeatures], learning_rate=0.1):
    y = x
    #encoders
    weights_1 = tf.Variable(tf.random_normal([NumFeatures, layer_size[0]]))
    bias_1 = tf.Variable(tf.random_normal([layer_size[0]]))
    weights_2 = tf.Variable(tf.random_normal([layer_size[0], layer_size[1]]))
    bias_2 = tf.Variable(tf.random_normal([layer_size[1]]))
    
    #decoders
    weights_3 = tf.Variable(tf.random_normal([layer_size[1], layer_size[2]]))
    bias_3 = tf.Variable(tf.random_normal([layer_size[2]]))
    weights_4 = tf.Variable(tf.random_normal([layer_size[2], NumFeatures]))
    bias_4 = tf.Variable(tf.random_normal([NumFeatures]))
    
    layer1 = tf.nn.relu(tf.matmul(x, weights_1) + bias_1)
    layer2 = tf.nn.relu(tf.matmul(layer1, weights_2) + bias_2)
    layer3 = tf.nn.relu(tf.matmul(layer2, weights_3) + bias_3)
    output = tf.nn.relu(tf.matmul(layer3, weights_4) + bias_4)
    
    cost = tf.reduce_mean(tf.pow(y-output, 2))
    momentum = 0.5
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum).minimize(cost)
    
    init = tf.global_variables_initializer()
    
    return init, optimizer     

In [30]:
with tf.Session() as sess:
    x = tf.placeholder(tf.float32, [None, NumFeatures])
    init, optimizer = construct(x)
    sess.run(init)
    numEpochs = 1000
    numBatches = 1000